#Taller con bases de datos NoSQL de "columnas anchas"/ "big columns", muy utilizadas en entornos de big data

---
1. **CASSANDRA** (Netflix, Apple con el Icloud):
[Cassandra, funcionamiento y principios](https://medium.com/@jarrietaalfaro/cassandra-funcionamiento-y-principios-926d5bebe576)
* Datos de series temporales, registros de actividad, aplicaciones IoT.
* Aplicaciones que requieren alta velocidad de escritura y disponibilidad
constante.
* Adecuados para aplicaciones con distribución geográfica.
2. **HBase** (Spotify y Adobe con Hadoop):
* Almacenamiento de datos estructurados para análisis en tiempo real.
* Aplicaciones que requieren acceso aleatorio y lecturas rápidas.
* Integración con el ecosistema Hadoop para procesamiento de big data.



# Identificación:
* Nombre: xxxxx
* correo: xxx
* asignatura: xxxxx

# 0.Instalar librerias

In [17]:
!pip install cassandra-driver==3.29.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 21.3 MB/s eta 0:00:00


# 0.Trabajando con google drive

In [1]:
# habilitamos drive de google desde colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.configuracion basica del proyecto

---
* credenciales
* llamado a proyecto de bigData (librerias propias)


## 1.1 IMPORTAR LIBRERIAS

In [43]:
import os
import sys
import pandas as pd
import importlib
#---------librerias para trabajar procesos en segundo plano (backGround)
import time
import subprocess
#------libreria para generar ID automaticos
import uuid

In [3]:
url_project='/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/BigData_project/'
sys.path.append(url_project+'Helpers')

In [5]:
from functions import functions
func = functions()

<<---Inializando class functions-->>


In [9]:
ruta_baseDatos='/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/CassandraDB'
func.crear_carpeta(ruta_baseDatos)

La carpeta ya existe: /content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/CassandraDB


# 2.Descargar/Instalar Cassandra DB  (JDK 11)

In [8]:
!java -version

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [7]:
#---------opcional sino tiene la versión 11 de jdk----------
!apt-get update
!apt-get install -y openjdk-11-jdk
!update-alternatives --config java

In [10]:
#cambiar el path del sistema operativo (para que trabaje en el directorio de la base de datos)
%cd {ruta_baseDatos}
#configurar variables de entorno
cassandra_home = f"{ruta_baseDatos}/apache-cassandra-4.1.4"
os.environ['CASSANDRA_HOME'] = cassandra_home
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['PATH'] += os.pathsep + f"{cassandra_home}/bin"

/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/CassandraDB


## 2.1 Descargar instalador y descomprimir

In [ ]:
!wget https://archive.apache.org/dist/cassandra/4.1.4/apache-cassandra-4.1.4-bin.tar.gz
!tar xzvf apache-cassandra-4.1.4-bin.tar.gz

## 2.2 Crear carpetas de datos

In [ ]:
func.crear_carpeta(cassandra_home+'/data')
func.crear_carpeta(cassandra_home+'/commitLog')
func.crear_carpeta(cassandra_home+'/save_changes')

## 2.3 Modificar configuración del servidor de CassandraDB

In [13]:
conf_file = f"{cassandra_home}/conf/cassandra.yaml"

# Actualizar rutas en el archivo de configuración
!sed -i "s|/var/lib/cassandra/data|{cassandra_home}/data|g" {conf_file}
!sed -i "s|/var/lib/cassandra/commitlog|{cassandra_home}/commitlog|g" {conf_file}
!sed -i "s|/var/lib/cassandra/saved_caches|{cassandra_home}/saved_caches|g" {conf_file}

In [14]:
# Configuración adicional para Colab
cassandra_env_sh = f"{cassandra_home}/conf/cassandra-env.sh"
!echo 'JVM_OPTS="$JVM_OPTS -Djava.rmi.server.hostname=127.0.0.1"' >> {cassandra_env_sh}

# 3. Iniciar Apache Cassandra DB

## 3.1 en segundo plano

In [20]:
# Iniciar el servicio
cassandra_process = subprocess.Popen(
    [f"{cassandra_home}/bin/cassandra", "-R"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
print("esperar 40 segundos para que inicie el servidor")
time.sleep(40)

esperar 40 segundos para que inicie el servidor


## 3.2 Instanciar driver/libreria

In [18]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

## 3.3 Configurar la conexión a la BaseDeDatos

In [21]:
cluster= Cluster(['127.0.0.1'],
                 port=9042,
                 connect_timeout=60,
                 protocol_version=5)
session= cluster.connect()

## 3.4 Configurar la llave de keySpace (contenedor/base de datos)

In [22]:
session.execute("""
  CREATE KEYSPACE IF NOT EXISTS bigdata
  WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
  AND durable_writes = true
""")

In [23]:
session.set_keyspace('bigdata')

# 4. Creando una tabla

In [24]:
session.execute("""
CREATE TABLE IF NOT EXISTS usuarios (
        id UUID PRIMARY KEY,
        nombre TEXT,
        email TEXT,
        fecha_registro TIMESTAMP
    )WITH compaction = {
        'class': 'TimeWindowCompactionStrategy',
        'compaction_window_size': '7',
        'compaction_window_unit': 'DAYS'
    }
    AND bloom_filter_fp_chance = 0.01;
""")

## 4.1 verificar creación de tabla

In [40]:
rows =session.execute("""
SELECT table_name
FROM system_schema.tables
WHERE keyspace_name = 'bigdata';
""")
for row in rows:
    print(row.table_name)


hurtos
usuarios


In [41]:
rows =session.execute("""
SELECT table_name, column_name
FROM system_schema.columns
WHERE keyspace_name = 'bigdata';
""")
for row in rows:
    print(row)

Row(table_name='hurtos', column_name='armas_medios')
Row(table_name='hurtos', column_name='cantidad')
Row(table_name='hurtos', column_name='codigo_dane')
Row(table_name='hurtos', column_name='departamento')
Row(table_name='hurtos', column_name='fecha_hecho')
Row(table_name='hurtos', column_name='genero')
Row(table_name='hurtos', column_name='grupo_etario')
Row(table_name='hurtos', column_name='id')
Row(table_name='hurtos', column_name='municipio')
Row(table_name='hurtos', column_name='tipo_de_hurto')
Row(table_name='usuarios', column_name='email')
Row(table_name='usuarios', column_name='fecha_registro')
Row(table_name='usuarios', column_name='id')
Row(table_name='usuarios', column_name='nombre')


# 5.cargando CSV en un dataframe (panda)


---
https://www.datos.gov.co/Seguridad-y-Defensa/Reporte-Hurto-por-Modalidades-Polic-a-Nacional/9vha-vh9n/about_data


In [27]:
ruta_csv='/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/Reporte_Hurto_por_Modalidades_Polic_a_Nacional_20250313.csv'
df_hurtos= pd.read_csv(ruta_csv,sep=',')
df_hurtos.head()

<ipython-input-27-1cb2ca2e30ef>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hurtos= pd.read_csv(ruta_csv,sep=',')


,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,TIPO DE HURTO,CANTIDAD
0,ANTIOQUIA,MEDELLÍN (CT),5001000,ARMA DE FUEGO,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
1,ANTIOQUIA,COPACABANA,5212000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
2,ANTIOQUIA,MEDELLÍN (CT),5001000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
3,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1
4,VALLE,CALI (CT),76001000,LLAVE MAESTRA,1/01/2010,NO APLICA,NO APLICA,HURTO AUTOMOTORES,1


In [33]:
df_hurtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731615 entries, 0 to 731614
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   DEPARTAMENTO   731615 non-null  object        
 1   MUNICIPIO      731615 non-null  object        
 2   CODIGO_DANE    725124 non-null  Int64         
 3   ARMAS_MEDIOS   731615 non-null  object        
 4   FECHA_HECHO    731615 non-null  datetime64[ns]
 5   GENERO         731615 non-null  object        
 6   GRUPO_ETARIO   731615 non-null  object        
 7   TIPO_DE_HURTO  731615 non-null  object        
 8   CANTIDAD       731615 non-null  int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(6)
memory usage: 50.9+ MB


## 5.1 ETL sobre el dataFrame

In [29]:
# quitar espacios en los nombres de las columnas
df_hurtos.columns= [col.replace(' ','_') for col in df_hurtos.columns]
df_hurtos.columns

Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HECHO', 'GENERO', 'GRUPO_ETARIO', 'TIPO_DE_HURTO', 'CANTIDAD'],
      dtype='object')

In [30]:
# convertir columna CODIGO_DANE en numerico
df_hurtos['CODIGO_DANE']= pd.to_numeric(df_hurtos['CODIGO_DANE'],errors='coerce').astype('Int64')

In [32]:
# convertir columna FECHA_HECHO en campo fecha formato DD/MM/YYYY
df_hurtos['FECHA_HECHO']= pd.to_datetime(df_hurtos['FECHA_HECHO'],format='%d/%m/%Y',errors='coerce')

## 5.2 crear tabla basada en el esquema del DataFrame

In [34]:
schema =df_hurtos.dtypes.to_dict()
schema

{'DEPARTAMENTO': dtype('O'),
 'MUNICIPIO': dtype('O'),
 'CODIGO_DANE': Int64Dtype(),
 'ARMAS_MEDIOS': dtype('O'),
 'FECHA_HECHO': dtype('<M8[ns]'),
 'GENERO': dtype('O'),
 'GRUPO_ETARIO': dtype('O'),
 'TIPO_DE_HURTO': dtype('O'),
 'CANTIDAD': dtype('int64')}

In [39]:

create_table_query = f"""
    CREATE TABLE IF NOT EXISTS hurtos (id UUID PRIMARY KEY,
"""
for column_name, column_type in schema.items():
    if column_type == 'object':  # Suponiendo que los objetos son texto
        create_table_query += f"        {column_name} TEXT,\n"
    elif column_type == 'int64' or column_type == 'Int64':  # Tipo entero
      create_table_query += f"        {column_name} BIGINT,\n"
    elif column_type == 'datetime64[ns]':  # Tipo fecha y hora
        create_table_query += f"        {column_name} TIMESTAMP,\n"
    elif column_type == 'float64':  # tipo decimal
        create_table_query += f"        {column_name} DOUBLE,\n"
    else:
        create_table_query += f"        {column_name} TEXT,\n"  # Tipo por defecto si no se reconoce

create_table_query+= """
    )
    WITH compaction = {
        'class': 'TimeWindowCompactionStrategy',
        'compaction_window_size': '7',
        'compaction_window_unit': 'DAYS'
    }
    AND bloom_filter_fp_chance = 0.01;"""
# Ejecutar la consulta para crear la tabla (se debe adaptar al proceso de conexión a Cassandra previo)
session.execute(create_table_query)


## 5.3 dividir DataFrame en 4 partes (25%)

In [42]:
n = len(df_hurtos)
df1 = df_hurtos.iloc[:n//4]
df2 = df_hurtos.iloc[n//4:2*n//4]
df3 = df_hurtos.iloc[2*n//4:3*n//4]
df4 = df_hurtos.iloc[3*n//4:]

print("Tamaño de cada DataFrame:")
print("df1:", len(df1))
print("df2:", len(df2))
print("df3:", len(df3))
print("df4:", len(df4))

Tamaño de cada DataFrame:
df1: 182903
df2: 182904
df3: 182904
df4: 182904


In [49]:
def insertar_registro(session, df):
    for index, row in df.iterrows():
        id_registro = uuid.uuid4()  #generar un ID
        if pd.isnull(row['FECHA_HECHO'])==False and type(row['FECHA_HECHO'])==pd.Timestamp:
            row['FECHA_HECHO']=row['FECHA_HECHO'].strftime('%Y-%m-%d') #convertir DD/MM/YYYY -> YYYY-MM-DD
        else:
            row['FECHA_HECHO']=None
        query = f"INSERT INTO hurtos (id, {', '.join(df.columns)}) VALUES (%s, {', '.join(['%s'] * len(df.columns))})"
        values = [id_registro] + [None if pd.isnull(val) else val for val in list(row)]
        session.execute(query, values)
        print(f"Registro {index} insertado con ID: {id_registro}")

In [ ]:
insertar_registro(session,df2)

In [52]:
cluster.shutdown()